In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home2/kapilrk04/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/kapilrk04/cache'
os.environ['HF_DATASETS_CACHE']="/scratch/kapilrk04/cache"

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
device

device(type='cuda')

In [4]:
!pip install transformers datasets sentencepiece

In [5]:
!pip install accelerate -U

In [6]:
!pip install evaluate

In [7]:
!pip install pandas numpy

In [8]:
import pandas as pd
asnq_dev = pd.read_csv("/home2/kapilrk04/anlp_proj/data_sets/asnq/dev.tsv", sep="\t", names=["sentence1", "sentence2", "label"])

In [9]:
asnq_dev[:15]

,sentence1,sentence2,label
0,who issued gold coins for the first time in india,Jump up ^ `` Puranas or Punch - Marked Coins (...,1
1,most passing yards by nfl qb in a game,"71 : Kramer , Tommy Tommy Kramer 000000001985 ...",3
2,where does cortisol come from in the body,Psychoneuroendocrinology.,1
3,the rime of the ancient mariner albatross symb...,Contents ( hide ) 1 Film 2 Music 2.1 Musical 2...,1
4,who was it that described the structure of dna,If the DNA is twisted in the direction of the ...,1
5,who dies in the beginning of deathly hallows p...,Retrieved 20 March 2010 .,1
6,what is cost of first class mail stamp,Main article : United States Postal Service Se...,1
7,when did somewhere over the rainbow come out,In films and TV shows ( edit ) In the film Thi...,1
8,where did they film the game of thrones,The fifth season was also well received by cri...,1
9,what observation led to the theory of seafloor...,Older seafloor is therefore colder than new se...,1


In [10]:
pd.value_counts(asnq_dev["label"])

label
1    870404
3     29558
2     25814
4      4286
Name: count, dtype: int64

In [11]:
asnq_train = pd.read_csv("/home2/kapilrk04/anlp_proj/data_sets/asnq/train.tsv", sep="\t", names=["sentence1", "sentence2", "label"])

In [12]:
asnq_train[:15]

,sentence1,sentence2,label
0,what is the use of fn key in mac,It is typically found on laptops due to their ...,1
1,when did the ipod 7th generation come out,"Cupertino , California : Apple .",1
2,who dies in season 6 once upon a time,"^ Jump up to : Abrams , Natalie ( August 5 , 2...",1
3,who was the first avatar ever in the last airb...,Jeremy Zuckerman and Benjamin Wynn composed th...,1
4,how many games do they need to win to win the ...,p. 21 .,1
5,when did rufus wainwright record across the un...,"266 -- 67 Jump up ^ Sheff , p. 266 Jump up ^ K...",1
6,two core functions of the world health organiz...,"In 1977 , the first list of essential medicine...",1
7,when did adam smith published the wealth of na...,Boston Review .,1
8,how many english soldiers died in world war 2,Statistics published in Russia list civilian w...,1
9,what is the location of gateway of india,Wikipedia ® is a registered trademark of the W...,1


In [13]:
pd.value_counts(asnq_train["label"])

label
1    19446120
3      442140
2      428122
4       61186
Name: count, dtype: int64

In [22]:
trainNeg = asnq_train[asnq_train['label']==3].sample(frac=0.25)
trainNeg.loc[:,'label'] = 0
trainPos = asnq_train[asnq_train['label']==4]
trainPos.loc[:,'label'] = 1

train_set = pd.concat([trainNeg, trainPos])
train_set['idx'] = range(1, len(train_set) + 1)

In [23]:
train_set.head()

,sentence1,sentence2,label,idx
14043360,where is the big 10 basketball tournament held,Due to the Big East 's use of that venue for t...,0,1
9504059,explain the difference between recorded and li...,"Within the arts , music may be classified as a...",0,2
7160749,who won the women's 2017 us open,It was her first major and first LPGA Tour win...,0,3
5738895,when do you get your unrestricted license in m...,As some of the 80000 decals sold are for drive...,0,4
9696439,who has the most blocks in a nba game,"13 Johnson , George T. George T. Johnson San A...",0,5


In [16]:
pd.value_counts(train_set["label"])

label
0    110535
1     61186
Name: count, dtype: int64

In [19]:
devNeg = asnq_dev[asnq_dev['label']==3].sample(frac=0.25)
devNeg.loc[:,'label'] = 0
devPos = asnq_dev[asnq_dev['label']==4]
devPos.loc[:,'label'] = 1

dev_set = pd.concat([devNeg, devPos])
dev_set['idx'] = range(1, len(dev_set) + 1)

In [20]:
dev_set.head()

,sentence1,sentence2,label,idx
538496,where does no game no life anime end,"focusing on the character Izuna , ran from May...",0,1
110546,what is the longest panic at the disco song title,Alternate version featured on deluxe version o...,0,2
9637,who wrote trust and believe by keyshia cole,The heartfelt ballad reflects upon Cole 's pos...,0,3
623470,when were the beatles inducted into the rock a...,"Four Tops Renaldo `` Obie '' Benson , Abdul ``...",0,4
525264,what type of economic system was utilized in t...,In practice this limited the private sector to...,0,5


In [21]:
pd.value_counts(dev_set["label"])

label
0    7390
1    4286
Name: count, dtype: int64

In [24]:
from datasets import Dataset, load_dataset

train_dataset = Dataset.from_pandas(train_set)
dev_dataset = Dataset.from_pandas(dev_set)

In [27]:
train_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 171721
})

In [26]:
train_dataset = train_dataset.remove_columns('__index_level_0__')
dev_dataset = dev_dataset.remove_columns('__index_level_0__')

In [28]:
dev_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 11676
})

In [44]:
from transformers import AutoTokenizer

model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [45]:
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_dev_dataset = dev_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/171721 [00:00<?, ? examples/s]

Map:   0%|          | 0/11676 [00:00<?, ? examples/s]

In [46]:
encoded_dev_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 11676
})

In [47]:
encoded_train_dataset[:5]

{'sentence1': ['where is the big 10 basketball tournament held',
  'explain the difference between recorded and live music',
  "who won the women's 2017 us open",
  'when do you get your unrestricted license in michigan',
  'who has the most blocks in a nba game'],
 'sentence2': ["Due to the Big East 's use of that venue for their conference tournament , the Big Ten Tournament took place one week earlier than usual , ending the week before Selection Sunday .",
  'Within the arts , music may be classified as a performing art , a fine art or as an auditory art .',
  'It was her first major and first LPGA Tour win after 10 wins on the LPGA of Korea Tour .',
  'As some of the 80000 decals sold are for drivers with multiple cars , it is estimated that 75 % of provisional drivers ignore this law .',
  '13 Johnson , George T. George T. Johnson San Antonio Spurs 000000001981 - 02 - 24 - 0000 February 24 , 1981 Golden State Warriors 131 -- 126 28 9 9 0 No Yes 11 blocks in the second half , tyin

In [48]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
import numpy as np
from sklearn.metrics import average_precision_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    avg_prec_scores = []
    num_queries = len(labels)

    for i in range(num_queries):
        avg_prec_scores.append(average_precision_score(labels[i], predictions[i]))
    
    map_score = np.mean(avg_prec_scores)

    mrr_sum = 0

    for i in range(num_queries):
        mrr_sum += 1/(np.where(labels[i]==1)[0][0]+1)
    
    mrr_score = mrr_sum/num_queries

    return {
        "mAP" : map_score,
        "MRR" : mrr_score
    }

In [41]:
encoded_train_dataset[:15]

{'sentence1': ['where is the big 10 basketball tournament held',
  'explain the difference between recorded and live music',
  "who won the women's 2017 us open",
  'when do you get your unrestricted license in michigan',
  'who has the most blocks in a nba game',
  "who was the first singing brothers inducted into the rock 'n' roll hall of fame",
  'where did the idea of spongebob come from',
  'who did snow white fall in love with',
  'when did the first motorola flip phone come out',
  'when does jane the virgin season 4 episode 13 come out',
  'when was attack on titan season 1 released',
  'who wrote never be alone by shawn mendes',
  'who has the most figure skating olympic medals',
  'list of chapters in a book is called',
  'when does jake show up in hannah montana'],
 'sentence2': ["Due to the Big East 's use of that venue for their conference tournament , the Big Ten Tournament took place one week earlier than usual , ending the week before Selection Sunday .",
  'Within the 

In [50]:
model_name = "roberta-base"
batch_size = 16

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=9,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home2/kapilrk04/anaconda3/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home2/kapilrk04/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home2/kapilrk04/anaconda3/lib/python3.11/site-packages/transformers/models/roberta/modeling_roberta.py", line 1196, in forward
    outputs = self.roberta(
              ^^^^^^^^^^^^^
  File "/home2/kapilrk04/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home2/kapilrk04/anaconda3/lib/python3.11/site-packages/transformers/models/roberta/mo